In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, recall_score
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame
import numpy as np

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
# import libraries for visualizations

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [5]:
df = pd.read_excel (r'/Users/MaeganBoosten/Thesis_Project/Final/Original_SHAP.xlsx')
print (df)

     duration  credit_amount  intallment_rate  residence_since       age  \
0    0.256953       1.078480         0.918477        -0.765977 -0.751642   
1   -0.240857      -0.237923         0.918477         1.046987  2.590552   
2    0.505858      -0.248556         0.024147         0.140505 -1.279356   
3    0.256953       1.168508        -0.870183        -0.765977 -1.279356   
4    2.248194       1.165673         0.024147        -0.765977 -1.015499   
..        ...            ...              ...              ...       ...   
369  1.252574       1.757239        -0.870183        -0.765977 -0.839594   
370 -0.738668      -0.877693         0.918477         1.046987  1.535122   
371 -0.738668      -0.912429         0.918477        -0.765977  0.919455   
372  0.256953      -0.728473         0.918477         1.046987  0.215835   
373 -0.738668      -0.615760        -1.764514        -1.672459 -1.103451   

     number_of_existcr  number_of_dependents  telephon   foreign  \
0             1.027

In [6]:
#split dataset

In [7]:
x, y = df.drop('False Positive', axis=1), df['False Positive']
x.shape, y.shape

((374, 114), (374,))

In [8]:
#Oversampling
# summarize class distribution
print(Counter(y))
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(x, y)
# summarize class distribution
print(Counter(y_over))

Counter({0: 270, 1: 104})
Counter({1: 270, 0: 270})


In [9]:
x_train, x_test, y_train, y_test= train_test_split(X_over,y_over, test_size=.5, random_state=42)

In [10]:
x_train.shape, x_test.shape

((270, 114), (270, 114))

In [11]:
#Number of False Positives in y_train
sum(y_train)

136

In [12]:
#Number of False Positives in y_test
sum(y_test)

134

In [13]:
#Baseline for evaluation
y.value_counts(normalize=True)

0    0.721925
1    0.278075
Name: False Positive, dtype: float64

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, recall_score, classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics

In [15]:
def acc_val(TP, TN, FP, FN, data):    
    accuracy = (TP + TN) / float(TP + TN + FP + FN)
    recall = TP / float(FN + TP)
    specificity = TN / (TN + FP)
    false_positive_rate = FP / float(TN + FP)
    precision = TP / float(TP + FP)
    F1 = 2 * (precision * recall) / (precision + recall)
    print('The values for {} are:'.format(data))
    print('Accuracy: {} ,Recall: {} , Specificity: {} , False Positive Rate: {} , Precision: {}, F1 Score: {}'.format(accuracy, recall, specificity, false_positive_rate,
                                                                                                                   precision, F1))

In [16]:
#Model: DecisionTree
tree= DecisionTreeClassifier(random_state=42)
# fit model
tree.fit(x_train, y_train)
# predict
pred_test= tree.predict(x_test)

In [17]:
pred_test.shape, y_test.shape

((270,), (270,))

In [18]:
accuracy_score(y_test, pred_test)

0.837037037037037

In [19]:
confusion_matrix(y_test, tree.predict(x_test))

array([[105,  31],
       [ 13, 121]])

In [20]:
print(classification_report(y_test, tree.predict(x_test)))

              precision    recall  f1-score   support

           0       0.89      0.77      0.83       136
           1       0.80      0.90      0.85       134

    accuracy                           0.84       270
   macro avg       0.84      0.84      0.84       270
weighted avg       0.84      0.84      0.84       270



In [21]:
confusion = metrics.confusion_matrix(y_test, tree.predict(x_test))
print(confusion)
TP1 = confusion[1, 1]
TN1 = confusion[0, 0]
FP1 = confusion[0, 1]
FN1 = confusion[1, 0]
acc_val(TP1, TN1, FP1, FN1, 'Test Data')

[[105  31]
 [ 13 121]]
The values for Test Data are:
Accuracy: 0.837037037037037 ,Recall: 0.9029850746268657 , Specificity: 0.7720588235294118 , False Positive Rate: 0.22794117647058823 , Precision: 0.7960526315789473, F1 Score: 0.8461538461538461


In [22]:
roc_auc_score(y_test, tree.predict(x_test))

0.8375219490781388

In [23]:
#Tuning using gridsearch

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, recall_score, classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate

In [25]:
params= {'max_depth': np.arange(2,11), 'min_samples_leaf': np.arange(2,11)  }

In [26]:
skf= StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [27]:
best_tree= GridSearchCV(tree, param_grid= params, cv=skf, n_jobs=-1, verbose=1)
best_tree

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'min_samples_leaf': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])},
             verbose=1)

In [28]:
params= {'max_depth': np.arange(2,11), 'min_samples_leaf': np.arange(2,11)  }
best_tree= GridSearchCV(tree, param_grid= params, cv=skf, n_jobs=-1, verbose=1)

In [29]:
best_tree.fit(x_train, y_train)


Fitting 10 folds for each of 81 candidates, totalling 810 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'min_samples_leaf': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])},
             verbose=1)

In [30]:
best_tree.best_params_


{'max_depth': 10, 'min_samples_leaf': 2}

In [31]:
best_tree.best_estimator_


DecisionTreeClassifier(max_depth=10, min_samples_leaf=2, random_state=42)

In [32]:
# cross-val assessment of model quality
best_tree.best_score_


0.7777777777777778

In [33]:
pred_test_better= best_tree.predict(x_test)

In [34]:
accuracy_score(y_test, best_tree.predict(x_test))

0.7888888888888889

In [35]:
confusion = metrics.confusion_matrix(y_test, best_tree.predict(x_test))
print(confusion)
TP1 = confusion[1, 1]
TN1 = confusion[0, 0]
FP1 = confusion[0, 1]
FN1 = confusion[1, 0]
acc_val(TP1, TN1, FP1, FN1, 'Test Data')

[[102  34]
 [ 23 111]]
The values for Test Data are:
Accuracy: 0.7888888888888889 ,Recall: 0.8283582089552238 , Specificity: 0.75 , False Positive Rate: 0.25 , Precision: 0.7655172413793103, F1 Score: 0.7956989247311828


In [36]:
roc_auc_score(y_test, best_tree.predict(x_test))

0.7891791044776119